In [31]:
import os
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset
import torchvision.transforms as transforms

# import keras
# from keras.models import Sequential
# from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from sklearn.metrics import accuracy_score

In [4]:
# Only for visualization of process
from tqdm import notebook
def tqdm(x, **kargs):
    return notebook.tqdm(x, leave=False, **kargs)

## Download dataset 
https://drive.google.com/drive/u/3/folders/1sHh6NvuKX6RB5OytLwf4kaqfQ9svJNDQ

## Load data

In [146]:
class_index = {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4,
               'dog': 5, 'frog': 6,'horse': 7,'ship': 8, 'truck': 9}

In [147]:
class customDataset(Dataset):
    def __init__(self, datatype, transform, classes):
        ##############################################
        ### Initialize paths, transforms, and so on
        ##############################################
        self.transform = transform
        self.images = np.load("./source/x_{}.npy".format(datatype))
        self.labels = np.load("./source/y_{}.npy".format(datatype))
        self.classes = classes
        assert len(self.images) == len(self.labels), 'mismatched length!'
        print("image shape: {}, label shape: {}".format(self.images.shape, self.labels.shape))
        
        
    def __getitem__(self, index):
        ##############################################
        # 1. Read from file (using numpy.fromfile, PIL.Image.open)
        # 2. Preprocess the data (torchvision.Transform).
        # 3. Return the data (e.g. image and label)
        ##############################################
        
        image = self.images[index]
        label = self.labels[index]
        
        if self.transform is not None:
            image = self.transform(image)
        return image, label
        
    def __len__(self):
        ##############################################
        ### Indicate the total size of the dataset
        ##############################################
        return len(self.images)

In [148]:
data_transforms = {
    'train': transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ]),
    'test': transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ]),
}

In [149]:
trainset = customDataset(datatype='train',
                         transform=data_transforms['train'],
                         classes=class_index)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True)

image shape: (50000, 32, 32, 3), label shape: (50000, 1)


In [150]:
testset = customDataset(datatype='test',
                        transform=data_transforms['test'],
                        classes=class_index)
testloader = torch.utils.data.DataLoader(testset, batch_size=8, shuffle=False)

image shape: (10000, 32, 32, 3), label shape: (10000, 1)


In [151]:
for imgs, lbls in trainloader:
    print('Size of image:', imgs.size())  
    print('Type of image:', imgs.dtype)   
    print('Size of label:', lbls.size())  
    print('Type of label:', lbls.dtype)
    
    print(imgs)
    print(lbls)
    break

Size of image: torch.Size([8, 3, 32, 32])
Type of image: torch.float32
Size of label: torch.Size([8, 1])
Type of label: torch.int64
tensor([[[[-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
          [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
          [-2.4291, -2.4291, -2.4291,  ..., -2.4291, -2.4291, -2.4291],
          ...,
          [-1.1884,  0.1297,  1.3704,  ..., -1.1690, -1.4211, -1.5374],
          [-1.0527, -0.6650,  0.0328,  ..., -1.4211, -1.4598, -1.5180],
          [-1.2660, -1.0140, -0.9558,  ..., -1.6343, -1.4792, -1.3629]],

         [[-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
          [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
          [-2.4183, -2.4183, -2.4183,  ..., -2.4183, -2.4183, -2.4183],
          ...,
          [-0.9236,  0.1974,  1.2594,  ..., -1.0022, -1.1596, -1.3169],
          [-0.5892, -0.3336,  0.2171,  ..., -1.1989, -1.1596, -1.2579],
          [-0.7072, -0.5499, -0.5499,  ..., 

In [152]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Build model & training (Pytorch)

In [153]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [154]:
net = Net()
net.to(device)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [155]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)

In [156]:
epochs = 2

In [157]:
for epoch in tqdm(range(epochs), desc='Epochs: '):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in tqdm(enumerate(trainloader, 0), desc='Epochs: '):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        labels = labels.view(-1)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
#         if i % 2000 == 1999:    # print every 2000 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                   (epoch + 1, i + 1, running_loss / 2000))
#             running_loss = 0.0

print('Finished Training')

Finished Training


In [158]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [181]:
class_correct = [0 for _ in range(len(class_index))]
class_total = [0 for _ in range(len(class_index))]
y_pred = []
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        labels = labels.view(-1)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        y_pred.append(predicted.cpu().detach().numpy())
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of {} : {}%'.format(i, 100 * class_correct[i] / class_total[i]))

Accuracy of 0 : 47.895791583166336%
Accuracy of 1 : 70.16460905349794%
Accuracy of 2 : 35.2%
Accuracy of 3 : 38.113207547169814%
Accuracy of 4 : 38.10444874274661%
Accuracy of 5 : 32.10633946830266%
Accuracy of 6 : 68.7007874015748%
Accuracy of 7 : 64.04255319148936%
Accuracy of 8 : 71.83673469387755%
Accuracy of 9 : 59.29549902152642%


In [185]:
test = []
for i in range(len(y_pred)):
    for j in range(8):
        test.append(y_pred[i][j])

In [186]:
y_pred = test.copy()
y_pred = np.array(y_pred)

In [187]:
y_pred

array([9, 6, 5, ..., 9, 8, 9])

## Build model & training (Keras)

In [9]:
# Builde model
model = Sequential() # Sequential groups a linear stack of layers 
model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=x_train.shape[1:])) # Add Convolution layers
model.add(Activation('relu')) # Add Relu activation for non-linearity
model.add(Conv2D(filters=32, kernel_size=(3, 3))) # Add Convolution layers
model.add(Activation('relu')) # Add Relu activation for non-linearity
model.add(MaxPooling2D(pool_size=(4, 4))) # Add Max pooling to lower the sptail dimension

model.add(Flatten()) # Flatten the featuremaps
model.add(Dense(units=512)) # Add dense layer with 512 neurons
model.add(Activation('relu')) # Add Relu activation for non-linearity
model.add(Dense(units=num_classes)) # Add final output layer for 10 classes
model.add(Activation('softmax')) # Add softmax activation to transfer logits into probabilities

# initiate SGD optimizer
opt = keras.optimizers.SGD()

# Compile the model with loss function and optimizer, and evaluate with accuracy
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Setup some hyperparameters
batch_size = 32
epochs = 10

# Fit the data into model
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 24s 478us/step - loss: 1.8933 - acc: 0.3203 - val_loss: 1.5972 - val_acc: 0.4367
Epoch 2/10
50000/50000 [==============================] - 43s 867us/step - loss: 1.4593 - acc: 0.4818 - val_loss: 1.3575 - val_acc: 0.5240
Epoch 3/10
50000/50000 [==============================] - 34s 686us/step - loss: 1.2797 - acc: 0.5500 - val_loss: 1.2541 - val_acc: 0.5644
Epoch 4/10
50000/50000 [==============================] - 36s 720us/step - loss: 1.1661 - acc: 0.5923 - val_loss: 1.2103 - val_acc: 0.5730
Epoch 5/10
50000/50000 [==============================] - 37s 744us/step - loss: 1.0775 - acc: 0.6257 - val_loss: 1.0946 - val_acc: 0.6187
Epoch 6/10
50000/50000 [==============================] - 45s 908us/step - loss: 1.0061 - acc: 0.6518 - val_loss: 1.0458 - val_acc: 0.6334
Epoch 7/10
50000/50000 [==============================] - 29s 589us/step - loss: 0.9396 - acc: 0.6733 - val_loss: 0.

In [10]:
y_pred = model.predict(x_test)
print(y_pred.shape) # 10000 samples, each sample with probaility of 10 classes

(10000, 10)


In [11]:
y_pred[0] 

array([2.4284909e-03, 2.9643339e-01, 1.2415329e-03, 3.2413865e-03,
       1.1167271e-03, 1.0127937e-03, 6.8496183e-06, 3.5009726e-03,
       3.1431669e-03, 6.8787467e-01], dtype=float32)

In [12]:
np.argmax(y_pred[0]) # argmax to find the predict class with highest probability. 9=truck

9

In [13]:
y_pred = np.argmax(y_pred, axis=1)

## DO NOT MODIFY CODE BELOW!
**Please screen shot your results and post it on your report**

In [ ]:
y_pred = your_model.predict(x_test)

In [188]:
assert y_pred.shape == (10000,)

In [189]:
y_test = np.load("./source/y_test.npy")
print("Accuracy of my model on test set: ", accuracy_score(y_test, y_pred))

Accuracy of my model on test set:  0.522
